# Projection and design

In this tab, the results of the clinical trial (ongoing or ended) are used to design a next phase and particularly the sample size. For on going clinical trial, one can also find a Bayesian approach (section 2) that can be used to interim monitoring in clinical trials.

## 1. Analysis sample size

Based on the summary of [[1]](https://www.statsdirect.com/help/sample_size/survival.htm) of the works of [[2]](https://www.sciencedirect.com/science/article/abs/pii/019724569090005M) and [[3]](https://www.jstor.org/stable/2530299), a table of sample sizes is proposed with several two side type I ($\alpha$) and type II ($\beta$) error values. The usual values are $\alpha$ = 0.05  and $\beta$ = 0.2. Note that in the table (*rows*) are provided the power that corresponds to 1 - $\beta$.

The method used here is suitable for calculating sample sizes for studies that will be analyzed by the log-rank test.

In [1]:
import os
import sys
sys.path.append("C:\\Users\\Fabien Boux\\Code\\ClinLib")

from functions.config import Config
from clinlib.database import Database

with open('init.txt') as f:
    lines = f.readlines()
config = Config(lines[0])
config.read()

database = Database(config.get_value('database', section='PATH'), idlength=(int(config.get_value('id_length', section='OTHER')) if config.is_key('id_length') else 3))
database.add_resource({'metadata': os.path.join(config.get_value('database', section='PATH'), config.get_value('metadata', section='PATH'))})

import ipywidgets as widgets
from ipywidgets import interact, interactive
import matplotlib.pyplot as plt
from IPython.display import display

In [2]:
%matplotlib widget

from functions.table import sample_size_table

if not config.is_key('followup'):
    config.extract_config_values('followup')
followup = int(int(config.get_value('followup')) / (365/12))

metric = (config.get_value('volume_label', section='OTHER') if config.is_key('volume_label') else 'Volume')

def n_table(Event='OS', Model='Hazard Ratio (HR)', Followup=followup):
    if Model == 'Hazard Ratio (HR)':
        Model = 'HR'
    elif Model == 'median Survival Time (mST)':
        Model = 'mST'
    
    if Event == 'PFS (adjusted)':
        if config.is_key('visits'):
            tab = sample_size_table(database, event='PFS', criteria=Model, adjust_ipfs=True, visits=config.get_value('followup_visits'), followup_time=Followup, metric=metric)
    else:
        tab = sample_size_table(database, event=Event, criteria=Model, adjust_ipfs=False, followup_time=Followup, metric=metric)
    
    return tab

interact(n_table, Event=['OS', 'PFS', 'PFS (adjusted)'], Model=['Hazard Ratio (HR)','median Survival Time (mST)'], Followup=(0, round(followup*1.5), 1));

interactive(children=(Dropdown(description='Event', options=('OS', 'PFS', 'PFS (adjusted)'), value='OS'), Drop…

## 2. Interim monitoring

According to [[4]](https://pubmed.ncbi.nlm.nih.gov/24872363/): "Bayesian predictive probabilities can be used for interim monitoring of clinical trials to estimate the probability of observing a statistically significant treatment effect if the trial were to continue to its predefined maximum sample size".

The question we wish to answer here is: the trial is likely to reach a definitive conclusion (ie., statistically significant) by the end of the study? To answer this question, several approaches have been proposed. In this analysis, we adopt the point of view of [[5]](https://www.researchsquare.com/article/rs-930504/v1), ie. the assessment of study success (i.e. chance of success) using conditional power (CP), the predictive power of success (PPoS) and probability of success (PoS) in a general setting with normally distributed test statistics and normal prior (restricted to time-to-event endpoints).

CP is frequentist tool, whereas PoS and PPoS follow the Bayesian paradigm.
The difference of Bayesian approach over the frequentist approach is the way available knowledge on effect size is summarized: Bayesian measures summarize this knowledge as distribution of effect whereas frequentist measure makes the best guess about the effect size as a single value, and thereby frequentist measures may not be a good indicator of chance of success.

First analysis are carried out using conventional frequentist techniques: the number of future events is computed based on the frequency of events in each group in interim data. A Cox’s proportional hazard model is used to assess hazard ratio (HR).

In [3]:
%matplotlib widget

from functions.table import probability_of_success

if not config.is_key('followup'):
    config.extract_config_values('followup')
followup = int(int(config.get_value('followup')) / (365 / 12))

metric = (config.get_value('volume_label', section='OTHER') if config.is_key('volume_label') else 'Volume')

l = (~database.get_metadata(which='all').dropna(how='all')['Start'].isna()).sum() + 1
nb = int((config.get_value('nb_patients', section='OTHER') if config.is_key('nb_patients') else l))

def n_table(Event='OS', Patients=nb, Followup=followup):
    if Event == 'PFS (adjusted)':
        if config.is_key('visits'):
            tab = probability_of_success(database, Patients, event='PFS', adjust_ipfs=True, visits=config.get_value('followup_visits'), followup_time=Followup, metric=metric)
    else:
        tab = probability_of_success(database, Patients, event=Event, adjust_ipfs=False, followup_time=Followup, metric=metric)
    
    print("Probabilities of success of the clinical trial with {} patients are:".format(Patients))
    return tab

interact(n_table, Event=['OS', 'PFS', 'PFS (adjusted)'], Followup=(0, round(followup*1.5), 1), Patients=(l, 1000, 1));

interactive(children=(Dropdown(description='Event', options=('OS', 'PFS', 'PFS (adjusted)'), value='OS'), IntS…

Unlike the trial success that is constant, the clinical success (ie., observing a treatment effect size exceeding some threshold value that is often clinically meaningful) is a function of the threshold value.

In [5]:
%matplotlib widget

from functions.graph import probability_of_success_plot

if not config.is_key('followup'):
    config.extract_config_values('followup')
followup = int(int(config.get_value('followup')) / (365 / 12))

metric = (config.get_value('volume_label', section='OTHER') if config.is_key('volume_label') else 'Volume')

l = (~database.get_metadata(which='all').dropna(how='all')['Start'].isna()).sum() + 1
nb = int((config.get_value('nb_patients', section='OTHER') if config.is_key('nb_patients') else l))

def n_table(Event='OS', Patients=nb, Followup=followup):
    plt.close()
    if Event == 'PFS (adjusted)':
        if config.is_key('visits'):
            probability_of_success_plot(database, Patients, event='PFS', adjust_ipfs=True, visits=config.get_value('followup_visits'), followup_time=Followup, metric=metric)
    else:
        probability_of_success_plot(database, Patients, event=Event, adjust_ipfs=False, followup_time=Followup, metric=metric)
    
    plt.show()

interact(n_table, Event=['OS', 'PFS', 'PFS (adjusted)'], Followup=(0, round(followup*1.5), 1), Patients=(l, 1000, 1));

interactive(children=(Dropdown(description='Event', options=('OS', 'PFS', 'PFS (adjusted)'), value='OS'), IntS…